### ARIMA
##### AutoRegressive Integrated Moving Average
- ARIMA models are well-known for their simplicity and interpretability. They can capture various patterns in time series data, including trends and seasonality (when using SARIMA).
- ARIMA is effective for short-term forecasting when the data shows clear autocorrelations.
- It is widely used in academic and professional settings for demand forecasting, making it a reliable choice.
---
- It's a well-established method in time series forecasting, making it a good baseline model.
- It's relatively easy to interpret, which helps in explaining the model's decisions.
- It works well for short-term forecasts, which aligns with the 14-day requirement.

In [ ]:
# Libraries setup
import os
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error
from pmdarima import auto_arima
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings('ignore')

### Data Load Pipeline
The data load pipeline is almost the same as in EDA. For the purpose of demonstration, it is not a new module, but placed as a function.

In [ ]:
# Define paths
fpath_data = os.environ.get("FPATH_DATA")
fpath_dicts = os.environ.get("FPATH_DICTS")

def get_data_order_items():
    # Load order items data
    order_items = pd.read_csv(fpath_data+"order_items.csv")
    # Convert shipping_limit_date to timestamps
    order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'])
    # Lets load data regarding product category, drop redundant columns, check the data, and then join the data to order_items
    product_categories = pd.read_csv(fpath_data+"products.csv")
    product_categories = product_categories[["product_id", "product_category_name"]]
    # Drop nulls
    product_categories = product_categories.dropna()
    # Map categories to english ones
    categories_translation_data = pd.read_csv(fpath_data+"product_category_name_translation.csv")
    product_categories = pd.merge(product_categories, categories_translation_data, how='left', on='product_category_name')
    # Instead of dropping 13 rows w/o translation, use the same data as originally
    product_categories['product_category_name_english'] = product_categories['product_category_name_english'].fillna(product_categories['product_category_name'])
    # Join product category to orders
    order_items = pd.merge(order_items, product_categories, how='left', on='product_id')
    # I decided to drop those without any category prescribed as those might be incorrect data
    order_items = order_items.dropna(subset=['product_category_name'])
    # Join data regarding purchase timestamp - first, read data
    orders = pd.read_csv(fpath_data+'orders.csv')
    # Keep only relevant columns
    orders = orders[['order_id','order_purchase_timestamp']]
    # Check for nulls
    orders.isnull().sum()
    # Transform order_purchase_timestamp to a timestamp object
    orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
    # Join orders data via inner - we could do also left join, but it would need to double check for nulls
    order_items = pd.merge(order_items, orders, how='inner', on='order_id')
    # Drop redundant data, not related to this task
    # As this is just a demonstration, we won't be adding information such as holidays for a different set of countries the customers are from etc
    order_items = order_items.drop(columns=['freight_value','shipping_limit_date','product_category_name'])
    # Add a set of information regarding sale timestamp
    order_items['Purchase Date'] = pd.to_datetime(order_items['order_purchase_timestamp']).dt.floor('D')
    return order_items


In [ ]:
order_items = get_data_order_items()
order_items.shape

##### As ARIMA is used, excessive features engineering is not required. Therefore, at this stage, the dataframe is grouped, and sorted by.

In [ ]:
dataset = order_items.groupby(['product_category_name_english', 'Purchase Date']).agg(
    sold_products_quantity=('order_item_id', 'size')
    # total_sales_value=('price', 'sum')
).sort_values(by=['product_category_name_english', 'Purchase Date'], ascending=True).reset_index()
dataset

In [ ]:
# dataset[dataset['product_category_name_english']=='art'].tail(50)

In [ ]:
# dataset[dataset['product_category_name_english']=='agro_industry_and_commerce'].tail(50)

##### To avoid manual search of terms such as AR, MA and checking differentiation, I use auto arima

In [ ]:
def test_stationarity(df):
    # Perform Dickey-Fuller test
    result = adfuller(df, autolag='AIC')
    return result[1] <= 0.05  # Return True if p-value <= 0.05 (stationary)

def find_optimal_d(df):
    d = 0
    while not test_stationarity(df) and d < 2:
        df = df.diff().dropna()
        d += 1
    return d

def analyze_acf_pacf(df, d):
    diff_series = df.diff(d).dropna()
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    plot_acf(diff_series, ax=ax1, lags=40)
    plot_pacf(diff_series, ax=ax2, lags=40)
    plt.show()

In [ ]:
def wmae(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred) * np.arange(1, len(y_true)+1)) / np.sum(np.arange(1, len(y_true)+1))

def bias(y_true, y_pred):
    return np.mean(y_pred - y_true)

In [ ]:
# Initialize dictionary to store forecasts and metrics to evaluate model
forecasts = {}
metrics = {}

# Get unique categories
categories = dataset['product_category_name_english'].unique()

In [ ]:
def sarima_forecast(train_data, test_data, forecast_end='2018-09-24'):
    print(f"Train data shape: {train_data.shape}")
    print(f"Test data shape: {test_data.shape}")
    print(f"Train data range: {train_data.min()} to {train_data.max()}")
    
    if train_data.empty or train_data.isnull().all():
        print("Error: Train data is empty or all null")
        return None, None

    if train_data.nunique() == 1:
        print(f"Warning: All values in train data are the same ({train_data.iloc[0]}). Using naive forecast.")
        constant_value = train_data.iloc[0]
        forecast_start = test_data.index[-1] + pd.Timedelta(days=1)
        forecast_dates = pd.date_range(start=forecast_start, end=forecast_end, freq='D')
        return (np.full(len(test_data), constant_value), 
                pd.Series(np.full(len(forecast_dates), constant_value), index=forecast_dates))

    try:
        model = auto_arima(train_data,
                        seasonal=True,
                        m=7,  # 7 or 365 for yearly seasonality
                        # start_p=0, start_q=0,
                        # max_p=5, max_q=5,
                        # start_P=0, start_Q=0,
                        # max_P=2, max_Q=2,
                        # d=None, D=None,
                        trace=True,
                        error_action='ignore',
                        suppress_warnings=True,
                        stepwise=True, # Exhaustive search
                        n_fits=100) # Increase for more thorough search        
        print(f"Best SARIMA order: {model.order}")
        print(f"Best seasonal order: {model.seasonal_order}")
       
        results = model.fit(train_data)
       
        in_sample_forecast = results.predict(n_periods=len(test_data))
        print(f"In-sample forecast shape: {in_sample_forecast.shape}")
        print(f"In-sample forecast range: {in_sample_forecast.min()} to {in_sample_forecast.max()}")
       
        forecast_start = test_data.index[-1] + pd.Timedelta(days=1)
        forecast_dates = pd.date_range(start=forecast_start, end=forecast_end, freq='D')
        out_of_sample_forecast = results.predict(n_periods=len(forecast_dates))
        print(f"Out-of-sample forecast shape: {out_of_sample_forecast.shape}")
        print(f"Out-of-sample forecast range: {out_of_sample_forecast.min()} to {out_of_sample_forecast.max()}")
       
        return in_sample_forecast, pd.Series(out_of_sample_forecast, index=forecast_dates)
    
    except Exception as e:
        print(f"Error during forecasting: {str(e)}")
        mean_value = train_data.mean()
        forecast_start = test_data.index[-1] + pd.Timedelta(days=1)
        forecast_dates = pd.date_range(start=forecast_start, end=forecast_end, freq='D')
        return (np.full(len(test_data), mean_value), 
                pd.Series(np.full(len(forecast_dates), mean_value), index=forecast_dates))

In [ ]:
for category in tqdm(categories, desc="Processing categories"):
    print(f"\nAnalyzing category: {category}")
   
    category_data = dataset[dataset['product_category_name_english'] == category]
    daily_sales = category_data.groupby('Purchase Date')['sold_products_quantity'].sum()
    daily_sales.index = pd.to_datetime(daily_sales.index)
    daily_sales = daily_sales.resample('D').sum()
   
    # Fill missing dates with 0 up to the last date in the dataset
    date_range = pd.date_range(start=daily_sales.index.min(), end=daily_sales.index.max(), freq='D')
    daily_sales = daily_sales.reindex(date_range, fill_value=0)
   
    split_point = int(len(daily_sales) * 0.8)
    train_data = daily_sales[:split_point]
    test_data = daily_sales[split_point:]
   
    in_sample_forecast, out_of_sample_forecast = sarima_forecast(train_data, test_data, forecast_end='2018-09-24')
   
    if in_sample_forecast is None or out_of_sample_forecast is None:
        print(f"Skipping category {category} due to forecast failure")
        continue

    # Replace NaN with mean value
    mean_value = train_data.mean()
    in_sample_forecast = np.nan_to_num(in_sample_forecast, nan=mean_value)
    out_of_sample_forecast = out_of_sample_forecast.fillna(mean_value)

    mae = mean_absolute_error(test_data, in_sample_forecast)
    wmae_score = wmae(test_data.values, in_sample_forecast)
    bias_score = bias(test_data.values, in_sample_forecast)
   
    metrics[category] = {
        'MAE': mae,
        'WMAE': wmae_score,
        'Bias': bias_score
    }
   
    forecasts[category] = out_of_sample_forecast

# Create DataFrames with all forecasts and metrics
forecast_df = pd.DataFrame(forecasts)
metrics_df = pd.DataFrame(metrics).T

print("\nForecasts up to 2018-09-24:")
print(forecast_df)
print("\nMetrics for each category:")
print(metrics_df)

forecast_df.to_csv('ARIMA_category_demand_forecasts.csv')
metrics_df.to_csv('ARIMA_category_forecast_metrics.csv')
print("Forecasts have been saved to 'ARIMA_category_demand_forecasts.csv'")
print("Metrics have been saved to 'ARIMA_category_forecast_metrics.csv'")